In [4]:
#Import the necessary modules 
import numpy as np
import matplotlib.pyplot as plt
import pandas
import math
from keras.models import Sequential
from keras.layers import Dense, Flatten
from keras.layers import LSTM
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
import datetime
from keras.layers import Embedding
from keras.models import load_model

Using TensorFlow backend.


In [5]:
# Tolerance Limit for computing the accuracy
tolerance_limit = 5
# Device count for determining the number of devices to extract the data 
deviceCount = 4
# Metric count for determining the count of the metric to perform the model computations 
# metricCount = 2

In [6]:
#Read the csv file as the input 
df = pandas.read_csv('data/output/output_events.csv', engine='python')
incidents = pandas.read_csv('data/output/output_incidents.csv', engine='python')

In [7]:
incidents

,INCIDENT_ID,SUBMIT_DATE,SUBMIT_DATE_MS,BUS_ID,EVENT_CODE,RECON_ID,EVENT_CODE_ID
0,INC001000003700,17-Aug-2018 17:09:59,1534525799,35711,ESM-12,OI-23F36A709CF011E8B954005056AC8828,NaN
1,INC001000003698,17-Aug-2018 17:09:58,1534525798,35711,ESM-12,OI-23F36A709CF011E8B954005056AC8828,NaN
2,INC001000704091,1-Feb-2020 10:06:44,1580551604,35711,EXCH-38,OI-23F36A709CF011E8B954005056AC8828,NaN
3,INC001000499761,28-Mar-2019 02:34:09,1553740449,34807,NLB-01,OI-210640589CF011E8B6B8005056AC8828,23.0
4,INC001000526149,26-Apr-2019 03:32:11,1556249531,34807,NLB-01,OI-210640589CF011E8B6B8005056AC8828,23.0
...,...,...,...,...,...,...,...
166,INC001000059374,28-Nov-2018 06:49:37,1543387777,35109,Storage,OI-473F7D529CF011E8BCF7005056AC8828,NaN
167,INC001000571403,9-Aug-2019 16:52:24,1565369544,35109,BCKP-11,OI-473F7D529CF011E8BCF7005056AC8828,NaN
168,INC001000499774,28-Mar-2019 02:39:11,1553740751,34807,NTWK-57,OI-210640589CF011E8B6B8005056AC8828,NaN
169,INC001000520128,13-Apr-2019 11:22:17,1555154537,34807,NTWK-57,OI-210640589CF011E8B6B8005056AC8828,NaN


In [8]:
df

,CORE_TIME,TARGET_ID,POLICY_ID,POLICY_TYPE,MEASURED_VALUE,EVENT_CODE,METRIC_NAME,RECON_ID
0,1567408585,1026606,1060,A,0,ISM-30,None,OI-473F7D529CF011E8BCF7005056AC8828
1,1567408885,1026606,1060,A,0,ISM-30,None,OI-473F7D529CF011E8BCF7005056AC8828
2,1567409185,1026606,1060,A,0,ISM-30,None,OI-473F7D529CF011E8BCF7005056AC8828
3,1567409485,1026606,1060,A,0,ISM-30,None,OI-473F7D529CF011E8BCF7005056AC8828
4,1567409785,1026606,1060,A,0,ISM-30,None,OI-473F7D529CF011E8BCF7005056AC8828
...,...,...,...,...,...,...,...,...
3475,1567493642,1667477,1240,A,1901,ISM-50,None,OI-23F36A709CF011E8B954005056AC8828
3476,1567493942,1667477,1240,A,1901,ISM-50,None,OI-23F36A709CF011E8B954005056AC8828
3477,1567494242,1667477,1240,A,1901,ISM-50,None,OI-23F36A709CF011E8B954005056AC8828
3478,1567494542,1667477,1240,A,1901,ISM-50,None,OI-23F36A709CF011E8B954005056AC8828


In [10]:
merged = incidents.merge(df, on=['EVENT_CODE','RECON_ID'], how="inner")

In [11]:
true_events = merged['RECON_ID'].unique()

In [12]:
true_events

array(['OI-210640589CF011E8B6B8005056AC8828',
       'OI-23F36A709CF011E8B954005056AC8828'], dtype=object)

In [14]:
BUFFER_MIN = 7*60*60
dataset_core = merged[(merged['SUBMIT_DATE_MS'] >= merged['CORE_TIME'] + BUFFER_MIN)]

In [15]:
dataset_core

,INCIDENT_ID,SUBMIT_DATE,SUBMIT_DATE_MS,BUS_ID,EVENT_CODE,RECON_ID,EVENT_CODE_ID,CORE_TIME,TARGET_ID,POLICY_ID,POLICY_TYPE,MEASURED_VALUE,METRIC_NAME
9715,INC001000688971,11-Jan-2020 10:27:45,1578738465,35711,ISM-50,OI-23F36A709CF011E8B954005056AC8828,20.0,1567411668,1667475,1240,A,1901,None
9716,INC001000688971,11-Jan-2020 10:27:45,1578738465,35711,ISM-50,OI-23F36A709CF011E8B954005056AC8828,20.0,1567411968,1667475,1240,A,1901,None
9717,INC001000688971,11-Jan-2020 10:27:45,1578738465,35711,ISM-50,OI-23F36A709CF011E8B954005056AC8828,20.0,1567412268,1667475,1240,A,1901,None
9718,INC001000688971,11-Jan-2020 10:27:45,1578738465,35711,ISM-50,OI-23F36A709CF011E8B954005056AC8828,20.0,1567412568,1667475,1240,A,1901,None
9719,INC001000688971,11-Jan-2020 10:27:45,1578738465,35711,ISM-50,OI-23F36A709CF011E8B954005056AC8828,20.0,1567412868,1667475,1240,A,1901,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...
37550,INC001000688970,11-Jan-2020 10:27:45,1578738465,35711,ISM-50,OI-23F36A709CF011E8B954005056AC8828,20.0,1567493642,1667477,1240,A,1901,None
37551,INC001000688970,11-Jan-2020 10:27:45,1578738465,35711,ISM-50,OI-23F36A709CF011E8B954005056AC8828,20.0,1567493942,1667477,1240,A,1901,None
37552,INC001000688970,11-Jan-2020 10:27:45,1578738465,35711,ISM-50,OI-23F36A709CF011E8B954005056AC8828,20.0,1567494242,1667477,1240,A,1901,None
37553,INC001000688970,11-Jan-2020 10:27:45,1578738465,35711,ISM-50,OI-23F36A709CF011E8B954005056AC8828,20.0,1567494542,1667477,1240,A,1901,None


In [16]:
# Device Ids to extract the data 
# step 1: choose one client who has the most number of devices
# step 2: choose all devices
device_ids = ['OI-210640589CF011E8B6B8005056AC8828', 'OI-23F36A709CF011E8B954005056AC8828', 'OI-473F7D529CF011E8BCF7005056AC8828']

In [17]:
dataset_core.count()

INCIDENT_ID       9280
SUBMIT_DATE       9280
SUBMIT_DATE_MS    9280
BUS_ID            9280
EVENT_CODE        9280
RECON_ID          9280
EVENT_CODE_ID     9280
CORE_TIME         9280
TARGET_ID         9280
POLICY_ID         9280
POLICY_TYPE       9280
MEASURED_VALUE    9280
METRIC_NAME       9280
dtype: int64

In [18]:
df

,CORE_TIME,TARGET_ID,POLICY_ID,POLICY_TYPE,MEASURED_VALUE,EVENT_CODE,METRIC_NAME,RECON_ID
0,1567408585,1026606,1060,A,0,ISM-30,None,OI-473F7D529CF011E8BCF7005056AC8828
1,1567408885,1026606,1060,A,0,ISM-30,None,OI-473F7D529CF011E8BCF7005056AC8828
2,1567409185,1026606,1060,A,0,ISM-30,None,OI-473F7D529CF011E8BCF7005056AC8828
3,1567409485,1026606,1060,A,0,ISM-30,None,OI-473F7D529CF011E8BCF7005056AC8828
4,1567409785,1026606,1060,A,0,ISM-30,None,OI-473F7D529CF011E8BCF7005056AC8828
...,...,...,...,...,...,...,...,...
3475,1567493642,1667477,1240,A,1901,ISM-50,None,OI-23F36A709CF011E8B954005056AC8828
3476,1567493942,1667477,1240,A,1901,ISM-50,None,OI-23F36A709CF011E8B954005056AC8828
3477,1567494242,1667477,1240,A,1901,ISM-50,None,OI-23F36A709CF011E8B954005056AC8828
3478,1567494542,1667477,1240,A,1901,ISM-50,None,OI-23F36A709CF011E8B954005056AC8828


In [19]:
match_value = []

# step 3: change to event id!!!
for i in range(0, len(df)):
    row = df.iloc[i, 7]
    if row in true_events:
        match_value.append(0)
    else:
        match_value.append(1)

In [20]:
match_value

[1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,


In [21]:
df.insert(8,'IS_NOISE',match_value)

In [22]:
df

,CORE_TIME,TARGET_ID,POLICY_ID,POLICY_TYPE,MEASURED_VALUE,EVENT_CODE,METRIC_NAME,RECON_ID,IS_NOISE
0,1567408585,1026606,1060,A,0,ISM-30,None,OI-473F7D529CF011E8BCF7005056AC8828,1
1,1567408885,1026606,1060,A,0,ISM-30,None,OI-473F7D529CF011E8BCF7005056AC8828,1
2,1567409185,1026606,1060,A,0,ISM-30,None,OI-473F7D529CF011E8BCF7005056AC8828,1
3,1567409485,1026606,1060,A,0,ISM-30,None,OI-473F7D529CF011E8BCF7005056AC8828,1
4,1567409785,1026606,1060,A,0,ISM-30,None,OI-473F7D529CF011E8BCF7005056AC8828,1
...,...,...,...,...,...,...,...,...,...
3475,1567493642,1667477,1240,A,1901,ISM-50,None,OI-23F36A709CF011E8B954005056AC8828,0
3476,1567493942,1667477,1240,A,1901,ISM-50,None,OI-23F36A709CF011E8B954005056AC8828,0
3477,1567494242,1667477,1240,A,1901,ISM-50,None,OI-23F36A709CF011E8B954005056AC8828,0
3478,1567494542,1667477,1240,A,1901,ISM-50,None,OI-23F36A709CF011E8B954005056AC8828,0


In [23]:
df['RECON_ID'].unique()

array(['OI-473F7D529CF011E8BCF7005056AC8828',
       'OI-23F36A709CF011E8B954005056AC8828',
       'OI-210640589CF011E8B6B8005056AC8828'], dtype=object)

In [24]:
df[df['IS_NOISE'] == 0].count()

CORE_TIME         2900
TARGET_ID         2900
POLICY_ID         2900
POLICY_TYPE       2900
MEASURED_VALUE    2900
EVENT_CODE        2900
METRIC_NAME       2900
RECON_ID          2900
IS_NOISE          2900
dtype: int64

In [26]:
new_data = {'RECON_ID': [], 'IS_NOISE': []}
df_data = pandas.DataFrame(new_data)

# step 4: metric name + normalized metric value + (device id)
for row in range(0,len(df)):
    new_row = []
    new_row.append(df.iloc[row]['RECON_ID'])
    new_row.append(df.iloc[row]['IS_NOISE'])
    print(new_row)
    # Convert the pandas data frame to a pandas series
    ps = pandas.Series(new_row, index=['RECON_ID','IS_NOISE'])
    # Append the pandas series to a data frame 
    df_data = df_data.append(ps, ignore_index=True)

['OI-473F7D529CF011E8BCF7005056AC8828', 1]
['OI-473F7D529CF011E8BCF7005056AC8828', 1]
['OI-473F7D529CF011E8BCF7005056AC8828', 1]
['OI-473F7D529CF011E8BCF7005056AC8828', 1]
['OI-473F7D529CF011E8BCF7005056AC8828', 1]
['OI-473F7D529CF011E8BCF7005056AC8828', 1]
['OI-473F7D529CF011E8BCF7005056AC8828', 1]
['OI-473F7D529CF011E8BCF7005056AC8828', 1]
['OI-473F7D529CF011E8BCF7005056AC8828', 1]
['OI-473F7D529CF011E8BCF7005056AC8828', 1]
['OI-473F7D529CF011E8BCF7005056AC8828', 1]
['OI-473F7D529CF011E8BCF7005056AC8828', 1]
['OI-473F7D529CF011E8BCF7005056AC8828', 1]
['OI-473F7D529CF011E8BCF7005056AC8828', 1]
['OI-473F7D529CF011E8BCF7005056AC8828', 1]
['OI-473F7D529CF011E8BCF7005056AC8828', 1]
['OI-473F7D529CF011E8BCF7005056AC8828', 1]
['OI-473F7D529CF011E8BCF7005056AC8828', 1]
['OI-473F7D529CF011E8BCF7005056AC8828', 1]
['OI-473F7D529CF011E8BCF7005056AC8828', 1]
['OI-473F7D529CF011E8BCF7005056AC8828', 1]
['OI-473F7D529CF011E8BCF7005056AC8828', 1]
['OI-473F7D529CF011E8BCF7005056AC8828', 1]
['OI-473F7D

['OI-473F7D529CF011E8BCF7005056AC8828', 1]
['OI-473F7D529CF011E8BCF7005056AC8828', 1]
['OI-473F7D529CF011E8BCF7005056AC8828', 1]
['OI-473F7D529CF011E8BCF7005056AC8828', 1]
['OI-473F7D529CF011E8BCF7005056AC8828', 1]
['OI-473F7D529CF011E8BCF7005056AC8828', 1]
['OI-473F7D529CF011E8BCF7005056AC8828', 1]
['OI-473F7D529CF011E8BCF7005056AC8828', 1]
['OI-473F7D529CF011E8BCF7005056AC8828', 1]
['OI-473F7D529CF011E8BCF7005056AC8828', 1]
['OI-473F7D529CF011E8BCF7005056AC8828', 1]
['OI-473F7D529CF011E8BCF7005056AC8828', 1]
['OI-473F7D529CF011E8BCF7005056AC8828', 1]
['OI-473F7D529CF011E8BCF7005056AC8828', 1]
['OI-473F7D529CF011E8BCF7005056AC8828', 1]
['OI-473F7D529CF011E8BCF7005056AC8828', 1]
['OI-473F7D529CF011E8BCF7005056AC8828', 1]
['OI-473F7D529CF011E8BCF7005056AC8828', 1]
['OI-473F7D529CF011E8BCF7005056AC8828', 1]
['OI-473F7D529CF011E8BCF7005056AC8828', 1]
['OI-473F7D529CF011E8BCF7005056AC8828', 1]
['OI-473F7D529CF011E8BCF7005056AC8828', 1]
['OI-473F7D529CF011E8BCF7005056AC8828', 1]
['OI-473F7D

['OI-23F36A709CF011E8B954005056AC8828', 0]
['OI-23F36A709CF011E8B954005056AC8828', 0]
['OI-23F36A709CF011E8B954005056AC8828', 0]
['OI-23F36A709CF011E8B954005056AC8828', 0]
['OI-23F36A709CF011E8B954005056AC8828', 0]
['OI-23F36A709CF011E8B954005056AC8828', 0]
['OI-23F36A709CF011E8B954005056AC8828', 0]
['OI-23F36A709CF011E8B954005056AC8828', 0]
['OI-23F36A709CF011E8B954005056AC8828', 0]
['OI-23F36A709CF011E8B954005056AC8828', 0]
['OI-23F36A709CF011E8B954005056AC8828', 0]
['OI-23F36A709CF011E8B954005056AC8828', 0]
['OI-23F36A709CF011E8B954005056AC8828', 0]
['OI-23F36A709CF011E8B954005056AC8828', 0]
['OI-23F36A709CF011E8B954005056AC8828', 0]
['OI-23F36A709CF011E8B954005056AC8828', 0]
['OI-23F36A709CF011E8B954005056AC8828', 0]
['OI-23F36A709CF011E8B954005056AC8828', 0]
['OI-23F36A709CF011E8B954005056AC8828', 0]
['OI-23F36A709CF011E8B954005056AC8828', 0]
['OI-23F36A709CF011E8B954005056AC8828', 0]
['OI-23F36A709CF011E8B954005056AC8828', 0]
['OI-23F36A709CF011E8B954005056AC8828', 0]
['OI-23F36A

['OI-23F36A709CF011E8B954005056AC8828', 0]
['OI-23F36A709CF011E8B954005056AC8828', 0]
['OI-23F36A709CF011E8B954005056AC8828', 0]
['OI-23F36A709CF011E8B954005056AC8828', 0]
['OI-23F36A709CF011E8B954005056AC8828', 0]
['OI-23F36A709CF011E8B954005056AC8828', 0]
['OI-23F36A709CF011E8B954005056AC8828', 0]
['OI-23F36A709CF011E8B954005056AC8828', 0]
['OI-23F36A709CF011E8B954005056AC8828', 0]
['OI-23F36A709CF011E8B954005056AC8828', 0]
['OI-23F36A709CF011E8B954005056AC8828', 0]
['OI-23F36A709CF011E8B954005056AC8828', 0]
['OI-23F36A709CF011E8B954005056AC8828', 0]
['OI-23F36A709CF011E8B954005056AC8828', 0]
['OI-23F36A709CF011E8B954005056AC8828', 0]
['OI-23F36A709CF011E8B954005056AC8828', 0]
['OI-23F36A709CF011E8B954005056AC8828', 0]
['OI-23F36A709CF011E8B954005056AC8828', 0]
['OI-23F36A709CF011E8B954005056AC8828', 0]
['OI-23F36A709CF011E8B954005056AC8828', 0]
['OI-23F36A709CF011E8B954005056AC8828', 0]
['OI-23F36A709CF011E8B954005056AC8828', 0]
['OI-23F36A709CF011E8B954005056AC8828', 0]
['OI-23F36A

['OI-210640589CF011E8B6B8005056AC8828', 0]
['OI-210640589CF011E8B6B8005056AC8828', 0]
['OI-210640589CF011E8B6B8005056AC8828', 0]
['OI-210640589CF011E8B6B8005056AC8828', 0]
['OI-210640589CF011E8B6B8005056AC8828', 0]
['OI-210640589CF011E8B6B8005056AC8828', 0]
['OI-210640589CF011E8B6B8005056AC8828', 0]
['OI-210640589CF011E8B6B8005056AC8828', 0]
['OI-210640589CF011E8B6B8005056AC8828', 0]
['OI-210640589CF011E8B6B8005056AC8828', 0]
['OI-210640589CF011E8B6B8005056AC8828', 0]
['OI-210640589CF011E8B6B8005056AC8828', 0]
['OI-210640589CF011E8B6B8005056AC8828', 0]
['OI-210640589CF011E8B6B8005056AC8828', 0]
['OI-210640589CF011E8B6B8005056AC8828', 0]
['OI-210640589CF011E8B6B8005056AC8828', 0]
['OI-210640589CF011E8B6B8005056AC8828', 0]
['OI-210640589CF011E8B6B8005056AC8828', 0]
['OI-210640589CF011E8B6B8005056AC8828', 0]
['OI-210640589CF011E8B6B8005056AC8828', 0]
['OI-210640589CF011E8B6B8005056AC8828', 0]
['OI-210640589CF011E8B6B8005056AC8828', 0]
['OI-210640589CF011E8B6B8005056AC8828', 0]
['OI-210640

...

['OI-23F36A709CF011E8B954005056AC8828', 0]
['OI-23F36A709CF011E8B954005056AC8828', 0]
['OI-23F36A709CF011E8B954005056AC8828', 0]
['OI-23F36A709CF011E8B954005056AC8828', 0]
['OI-23F36A709CF011E8B954005056AC8828', 0]
['OI-23F36A709CF011E8B954005056AC8828', 0]
['OI-23F36A709CF011E8B954005056AC8828', 0]
['OI-23F36A709CF011E8B954005056AC8828', 0]
['OI-23F36A709CF011E8B954005056AC8828', 0]
['OI-23F36A709CF011E8B954005056AC8828', 0]
['OI-23F36A709CF011E8B954005056AC8828', 0]
['OI-23F36A709CF011E8B954005056AC8828', 0]
['OI-23F36A709CF011E8B954005056AC8828', 0]
['OI-23F36A709CF011E8B954005056AC8828', 0]
['OI-23F36A709CF011E8B954005056AC8828', 0]
['OI-23F36A709CF011E8B954005056AC8828', 0]
['OI-23F36A709CF011E8B954005056AC8828', 0]
['OI-23F36A709CF011E8B954005056AC8828', 0]
['OI-23F36A709CF011E8B954005056AC8828', 0]
['OI-23F36A709CF011E8B954005056AC8828', 0]
['OI-23F36A709CF011E8B954005056AC8828', 0]
['OI-23F36A709CF011E8B954005056AC8828', 0]
['OI-23F36A709CF011E8B954005056AC8828', 0]
['OI-23F36A

['OI-23F36A709CF011E8B954005056AC8828', 0]
['OI-23F36A709CF011E8B954005056AC8828', 0]
['OI-23F36A709CF011E8B954005056AC8828', 0]
['OI-23F36A709CF011E8B954005056AC8828', 0]
['OI-23F36A709CF011E8B954005056AC8828', 0]
['OI-23F36A709CF011E8B954005056AC8828', 0]
['OI-23F36A709CF011E8B954005056AC8828', 0]
['OI-23F36A709CF011E8B954005056AC8828', 0]
['OI-23F36A709CF011E8B954005056AC8828', 0]
['OI-23F36A709CF011E8B954005056AC8828', 0]
['OI-23F36A709CF011E8B954005056AC8828', 0]
['OI-23F36A709CF011E8B954005056AC8828', 0]
['OI-23F36A709CF011E8B954005056AC8828', 0]
['OI-23F36A709CF011E8B954005056AC8828', 0]
['OI-23F36A709CF011E8B954005056AC8828', 0]
['OI-23F36A709CF011E8B954005056AC8828', 0]
['OI-23F36A709CF011E8B954005056AC8828', 0]
['OI-23F36A709CF011E8B954005056AC8828', 0]
['OI-23F36A709CF011E8B954005056AC8828', 0]
['OI-23F36A709CF011E8B954005056AC8828', 0]
['OI-23F36A709CF011E8B954005056AC8828', 0]
['OI-23F36A709CF011E8B954005056AC8828', 0]
['OI-23F36A709CF011E8B954005056AC8828', 0]
['OI-23F36A

['OI-23F36A709CF011E8B954005056AC8828', 0]
['OI-23F36A709CF011E8B954005056AC8828', 0]
['OI-23F36A709CF011E8B954005056AC8828', 0]
['OI-23F36A709CF011E8B954005056AC8828', 0]
['OI-23F36A709CF011E8B954005056AC8828', 0]
['OI-23F36A709CF011E8B954005056AC8828', 0]
['OI-23F36A709CF011E8B954005056AC8828', 0]
['OI-23F36A709CF011E8B954005056AC8828', 0]
['OI-23F36A709CF011E8B954005056AC8828', 0]
['OI-23F36A709CF011E8B954005056AC8828', 0]
['OI-23F36A709CF011E8B954005056AC8828', 0]
['OI-23F36A709CF011E8B954005056AC8828', 0]
['OI-23F36A709CF011E8B954005056AC8828', 0]
['OI-23F36A709CF011E8B954005056AC8828', 0]
['OI-23F36A709CF011E8B954005056AC8828', 0]
['OI-23F36A709CF011E8B954005056AC8828', 0]
['OI-23F36A709CF011E8B954005056AC8828', 0]
['OI-23F36A709CF011E8B954005056AC8828', 0]
['OI-23F36A709CF011E8B954005056AC8828', 0]
['OI-23F36A709CF011E8B954005056AC8828', 0]
['OI-23F36A709CF011E8B954005056AC8828', 0]
['OI-23F36A709CF011E8B954005056AC8828', 0]
['OI-23F36A709CF011E8B954005056AC8828', 0]
['OI-23F36A

['OI-23F36A709CF011E8B954005056AC8828', 0]
['OI-23F36A709CF011E8B954005056AC8828', 0]
['OI-23F36A709CF011E8B954005056AC8828', 0]
['OI-23F36A709CF011E8B954005056AC8828', 0]
['OI-23F36A709CF011E8B954005056AC8828', 0]
['OI-23F36A709CF011E8B954005056AC8828', 0]
['OI-23F36A709CF011E8B954005056AC8828', 0]
['OI-23F36A709CF011E8B954005056AC8828', 0]
['OI-23F36A709CF011E8B954005056AC8828', 0]
['OI-23F36A709CF011E8B954005056AC8828', 0]
['OI-23F36A709CF011E8B954005056AC8828', 0]
['OI-23F36A709CF011E8B954005056AC8828', 0]
['OI-23F36A709CF011E8B954005056AC8828', 0]
['OI-23F36A709CF011E8B954005056AC8828', 0]
['OI-23F36A709CF011E8B954005056AC8828', 0]
['OI-23F36A709CF011E8B954005056AC8828', 0]
['OI-23F36A709CF011E8B954005056AC8828', 0]
['OI-23F36A709CF011E8B954005056AC8828', 0]
['OI-23F36A709CF011E8B954005056AC8828', 0]
['OI-23F36A709CF011E8B954005056AC8828', 0]
['OI-23F36A709CF011E8B954005056AC8828', 0]
['OI-23F36A709CF011E8B954005056AC8828', 0]
['OI-23F36A709CF011E8B954005056AC8828', 0]
['OI-23F36A

['OI-23F36A709CF011E8B954005056AC8828', 0]
['OI-23F36A709CF011E8B954005056AC8828', 0]
['OI-23F36A709CF011E8B954005056AC8828', 0]
['OI-23F36A709CF011E8B954005056AC8828', 0]
['OI-23F36A709CF011E8B954005056AC8828', 0]
['OI-23F36A709CF011E8B954005056AC8828', 0]
['OI-23F36A709CF011E8B954005056AC8828', 0]
['OI-23F36A709CF011E8B954005056AC8828', 0]
['OI-23F36A709CF011E8B954005056AC8828', 0]
['OI-23F36A709CF011E8B954005056AC8828', 0]
['OI-23F36A709CF011E8B954005056AC8828', 0]
['OI-23F36A709CF011E8B954005056AC8828', 0]
['OI-23F36A709CF011E8B954005056AC8828', 0]
['OI-23F36A709CF011E8B954005056AC8828', 0]
['OI-23F36A709CF011E8B954005056AC8828', 0]
['OI-23F36A709CF011E8B954005056AC8828', 0]
['OI-23F36A709CF011E8B954005056AC8828', 0]
['OI-23F36A709CF011E8B954005056AC8828', 0]
['OI-23F36A709CF011E8B954005056AC8828', 0]
['OI-23F36A709CF011E8B954005056AC8828', 0]
['OI-23F36A709CF011E8B954005056AC8828', 0]
['OI-23F36A709CF011E8B954005056AC8828', 0]
['OI-23F36A709CF011E8B954005056AC8828', 0]
['OI-23F36A

In [27]:
df_data[df_data['IS_NOISE'] == 0]

,RECON_ID,IS_NOISE
290,OI-23F36A709CF011E8B954005056AC8828,0.0
291,OI-23F36A709CF011E8B954005056AC8828,0.0
292,OI-23F36A709CF011E8B954005056AC8828,0.0
293,OI-23F36A709CF011E8B954005056AC8828,0.0
294,OI-23F36A709CF011E8B954005056AC8828,0.0
...,...,...
3475,OI-23F36A709CF011E8B954005056AC8828,0.0
3476,OI-23F36A709CF011E8B954005056AC8828,0.0
3477,OI-23F36A709CF011E8B954005056AC8828,0.0
3478,OI-23F36A709CF011E8B954005056AC8828,0.0


In [83]:
# step 5: change string to binary (dummy variable 01): device id + metric name
df_data.RECON_ID = pandas.Categorical(df_data.RECON_ID)

In [84]:
df_data['CODE'] = df_data.RECON_ID.cat.codes

In [85]:
df_data

,RECON_ID,IS_NOISE,code
0,OI-473F7D529CF011E8BCF7005056AC8828,1.0,2
1,OI-473F7D529CF011E8BCF7005056AC8828,1.0,2
2,OI-473F7D529CF011E8BCF7005056AC8828,1.0,2
3,OI-473F7D529CF011E8BCF7005056AC8828,1.0,2
4,OI-473F7D529CF011E8BCF7005056AC8828,1.0,2
...,...,...,...
3475,OI-23F36A709CF011E8B954005056AC8828,0.0,1
3476,OI-23F36A709CF011E8B954005056AC8828,0.0,1
3477,OI-23F36A709CF011E8B954005056AC8828,0.0,1
3478,OI-23F36A709CF011E8B954005056AC8828,0.0,1


In [86]:
df_data.iloc[0:5, 2]

0    2
1    2
2    2
3    2
4    2
Name: code, dtype: int8

In [87]:
# convert an array of values into a dataset matrix
# use normalization of metric value + metric name!!!
def create_dataset(dataset, future_day, past_day):
    """
    Function to create a dataset based on the look back 
    :dataset - Input dataset with the necessary metrics  
    """
    # Create the list to store the input data point and the output labels 
    dataX, dataY = [], []
    # Iterate over the dataset with the lookback 
    for i in range(0, len(dataset)-future_day-past_day):
        # Append the input data set 
        dataX.append(dataset.iloc[i:(i+past_day), 2])
        # Append the label 
        dataY.append(dataset.iloc[i:(i+past_day), 1])
    # Return the numpy array of the result 
    return np.array(dataX), np.array(dataY)

In [88]:
df_data.values

array([['OI-473F7D529CF011E8BCF7005056AC8828', 1.0, 2],
       ['OI-473F7D529CF011E8BCF7005056AC8828', 1.0, 2],
       ['OI-473F7D529CF011E8BCF7005056AC8828', 1.0, 2],
       ...,
       ['OI-23F36A709CF011E8B954005056AC8828', 0.0, 1],
       ['OI-23F36A709CF011E8B954005056AC8828', 0.0, 1],
       ['OI-23F36A709CF011E8B954005056AC8828', 0.0, 1]], dtype=object)

In [89]:
future_day = 0
past_day = 15
# Invoke create dataset function for the train dataset 
dataX, dataY = create_dataset(df_data, future_day, past_day)

In [90]:
dataX

array([[2, 2, 2, ..., 2, 2, 2],
       [2, 2, 2, ..., 2, 2, 2],
       [2, 2, 2, ..., 2, 2, 2],
       ...,
       [1, 1, 1, ..., 1, 1, 1],
       [1, 1, 1, ..., 1, 1, 1],
       [1, 1, 1, ..., 1, 1, 1]], dtype=int8)

In [91]:
dataY

array([[1., 1., 1., ..., 1., 1., 1.],
       [1., 1., 1., ..., 1., 1., 1.],
       [1., 1., 1., ..., 1., 1., 1.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [92]:
dataX[0]

array([2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2], dtype=int8)

In [93]:
dataX[0] = pandas.Categorical(dataX[0])

In [94]:
# split into train and test sets
train_size = int(len(dataX) * 0.8)

# Determine the testsize 
test_size = len(dataX) - train_size
print(train_size)
print(test_size)
# Obtain the train and the test size 
trainX = dataX[0:train_size]
trainY = dataY[0:train_size]
testX = dataX[train_size:len(df)]
testY = dataY[train_size:len(df)]

2772
693


In [95]:
# Printing the values 
print(trainX[0])
print(trainY[0])
print(testX[0])
print(testY[0])

[2 2 2 2 2 2 2 2 2 2 2 2 2 2 2]
[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]


In [96]:
trainX

array([[2, 2, 2, ..., 2, 2, 2],
       [2, 2, 2, ..., 2, 2, 2],
       [2, 2, 2, ..., 2, 2, 2],
       ...,
       [1, 1, 1, ..., 1, 1, 1],
       [1, 1, 1, ..., 1, 1, 1],
       [1, 1, 1, ..., 1, 1, 1]], dtype=int8)

In [97]:
trainX = trainX.reshape(2772, 15, 1)
trainY = trainY.reshape(2772, 15, 1)

In [98]:
# create and fit the LSTM network using the Keras library 
model = Sequential()
model.add(LSTM(100, input_shape=(15,1), return_sequences=True))
# Add a fully connected layer at the end of the LSTM
model.add(Dense(1))
# Use the MSE loss and adam optimizer 
model.compile(loss='mse', optimizer='adam', metrics=['accuracy'])
print(model.summary())
max_eval = 100
count = 0
history = model.fit(trainX, trainY, epochs=10, verbose=3)
# # Train the model for 150 epochs 
# for ep in range(150):
#     print("Epoch Number : %s" %count)
#     count = count + 1
#     # For every epoch fit the train data 
# #     model.fit(trainX, trainY, epochs=1,batch_size=32, verbose=2)
#     model.fit(data, data, epochs=1,batch_size=32, verbose=2)
#     print(model.evaluate(testX,testY))
#     # Determine the test mse 
#     eval = model.evaluate(testX,testY)[0]
    # If the test mse is less than the previous mse
#     if(eval < max_eval):
#         # Save the model parameters 
#         print("Saving Model Parameter")
#         model.save('model_params.hd5')
#         max_eval = eval

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_2 (LSTM)                (None, 15, 100)           40800     
_________________________________________________________________
dense_2 (Dense)              (None, 15, 1)             101       
Total params: 40,901
Trainable params: 40,901
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/10
Epoch 2/10
Epoch 3/10
Epoch 4/10
Epoch 5/10
Epoch 6/10
Epoch 7/10
Epoch 8/10
Epoch 9/10
Epoch 10/10


In [102]:
testX = testX.reshape(693, 15, 1)
testY = testY.reshape(693, 15, 1)


In [103]:
model.evaluate(testX,testY)

693/693 [==============================] - 0s 451us/step


[0.0024863923758719907, 1.0]

In [104]:
model.predict(testX)

array([[[ 0.18411681],
        [ 0.04874946],
        [-0.02466564],
        ...,
        [-0.00196917],
        [-0.00018872],
        [ 0.00131668]],

       [[ 0.18411681],
        [ 0.04874946],
        [-0.02466564],
        ...,
        [-0.00196917],
        [-0.00018872],
        [ 0.00131668]],

       [[ 0.18411681],
        [ 0.04874946],
        [-0.02466564],
        ...,
        [-0.00196917],
        [-0.00018872],
        [ 0.00131668]],

       ...,

       [[ 0.18411681],
        [ 0.04874947],
        [-0.02466566],
        ...,
        [-0.00196917],
        [-0.00018865],
        [ 0.00131668]],

       [[ 0.18411681],
        [ 0.04874947],
        [-0.02466566],
        ...,
        [-0.00196917],
        [-0.00018865],
        [ 0.00131668]],

       [[ 0.18411681],
        [ 0.04874947],
        [-0.02466566],
        ...,
        [-0.00196919],
        [-0.00018867],
        [ 0.00131668]]], dtype=float32)

In [ ]:
# step 6: based on predicted value to get prob!!!